<a href="https://colab.research.google.com/github/flickmatch/home/blob/data-gen/highlights-ml/scripts/Goal_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Binary Classification Model for Detection Goals


In [1]:
import tensorflow as tf
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Splitting input data into train and validation sets

In [2]:
batch_size = 32
img_height = 224
img_width = 224
data_dir = '/content/gdrive/MyDrive/fm_goal_clf_data'

train_dataset = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

class_names = train_dataset.class_names

Found 14253 files belonging to 2 classes.
Using 11403 files for training.
Found 14253 files belonging to 2 classes.
Using 2850 files for validation.


In [3]:
class_names = train_dataset.class_names
print(class_names)

['goal', 'not_goal']


### Preprocess the Dataset

In [4]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))

# Test / Notice the pixel values are now in `[0,1]`.
# image_batch, labels_batch = next(iter(train_dataset))
# first_image = image_batch[0]
# print(np.min(first_image), np.max(first_image))

0.0 1.0


## Binary Classification Training Script based on CNNs

In [5]:
model = tf.keras.models.Sequential()

# VGG16
# model.add(tf.keras.applications.VGG16(
#     include_top=False,
#     weights='imagenet',
#     input_shape=(224, 224, 3),
#     pooling='avg'
# ))

# ResNet50
model.add(tf.keras.applications.resnet50.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3),
    pooling='avg'
))
model.add(tf.keras.layers.Dense(2))

model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

model.fit(
  train_dataset,
  validation_data=validation_dataset,
  epochs=3
)

94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/3
  8/357 [..............................] - ETA: 3:17:49 - loss: 1.1109 - accuracy: 0.6914

KeyboardInterrupt: ignored